In [ ]:
## import csv
import pandas as pd
import random
import re
import time
from tqdm import tqdm, trange
import numpy as np
import math
import codecs
import copy

In [2]:
def andor(Y,A):
    m = 2566
    YA = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            if Y[i][j] == 0 and A[i][j] == 0:
                YA[i][j] = 1
            elif Y[i][j] == 1 and A[i][j] == 1:
                YA[i][j] = 1
            else:
                YA[i][j] = 0               
    return YA

In [3]:
def plus(X):
    X1 = (abs(X)+X)/2
    return X1

In [4]:
def minus(X):
    X1 = (abs(X)-X)/2
    return X1

In [5]:
def OneZeroNorm(X):
    row_num = X.shape[0]
    col_num = X.shape[1]
    temp = X.sum(axis=1)#       
    #print temp     
    X1 = copy.deepcopy(X)
    for i in range(row_num):
        for j in range(col_num):
            if temp[i] != 0:
                X1[i][j] = X[i][j]/temp[i]
            else:
                X1[i][j] = 0
    return X1

In [6]:
def calL():
    m = 0
    r = 0

    f = open('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/authorpaper.txt')
    line = f.readline()
    line = line.replace(' ','').strip('\n')
    n_init = len(line)#文章数
    print(n_init)
    m = len(f.readlines()) + 1 #作者数
    print(m)
    f.close()

    r = 250   #假定已标记前895个
    n = 352   #除去core文章
    Wtemp = np.zeros((m,n_init))     #作者-文章矩阵
    f = open('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/authorpaper.txt')
    lines = f.readlines()
    Wtemp_row = 0
    for line in lines:
        list = line.strip().split(' ')
        Wtemp[Wtemp_row:] = list[0:n_init]
        Wtemp_row += 1

    W = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            if 0 <= j and j < 100:
                W[i][j] = Wtemp[i][j+38]
            elif 100 <= j and j < 250:
                W[i][j] = Wtemp[i][j+75]
            elif 250 <= j and j < 287:
                W[i][j] = Wtemp[i][j-112]
            else:
                W[i][j] = Wtemp[i][j+38]


    yL = np.ones((r,1))   #真假标签,如果是1则是假
    for i in range(r):
        if i>=0 and i < 100:
            yL[i][0] = -1
        else:
            yL[i][0] = 1
    
     #读入c
    f = open('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/authorcredit.txt')
    c = np.ones((m,1))
    lines = f.readlines()
    c_row = 0
    for line in lines:
        list = line.strip('\n')
        c[c_row] = list[0]
        c_row += 1

    f.close()
    
    G = np.ones((m,r))
    F = np.zeros((m+r,m+r))
    for i in range(m):
        for j in range(r):
            G[i][j] = W[i][j] * ( c[i][0] * ( 1 - ( 1 + yL[j][0] ) / 2 ) + ( 1 - c[i][0]) * ( 1 + yL[j][0]) / 2)

    for i in range(m+r):
        for j in range(m+r):
            if i>=0 and i < m and j >= m and j < m+r:
                F[i][j] = G[i][j-m]
            elif j>=0 and j < m and i >= m and i < m+r:
                F[i][j] = G[j][i-m]
                              
    S = np.zeros((m+r,m+r))
    F2 = F.sum(axis = 0)

    for i in range(m+r):
        S[i][i] = F2[i]

    L = S - F
    print('----------calculate L--------------')
    return L

In [7]:
def evaluate(yU):
    TP = 0  #真的，判断为真的数量
    FN = 0  #真的，判断为假的数量
    TN = 0  #假的，判断为假的数量
    FP = 0  #假的，判断为真的数量
    for i in range(102):
        if  0 <= i < 37:
            if yU[i] == -1:
                TP += 1
            else:
                FN += 1
        if  i >=37:
            if yU[i] == 1:
                TN += 1
            else:
                FP += 1
    print(TP)
    print(FP)
    print(TN)
    print(FN)
     
    Accuracy = (TP+TN)/float(TP+FP+TN+FN)
    Precision = TP/float(TP+FP)
    Recall = TP/float(TP+FN)
    F1 = 2*((Precision*Recall)/float(Precision+Recall))
    print("Accuracy："+str(Accuracy))
    print("Precision："+str(Precision))
    print("Recall："+str(Recall))
    print("F1："+str(F1))
    with codecs.open(r'/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/processed/valuate_add_credit.txt','a') as f:
        f.write(str(Accuracy)+'\t'+str(Precision)+'\t'+str(Recall)+'\t'+str(F1)+'\n')

In [8]:
n_init = 390#n+m+h'
n = 352 #1696个中前204个是core,m+h'
t = 128#128
r = 250 #从1492个中挑选出前654个T和前540个F, m和h'
m = 2566 #作者数
l = 102

gama = 10
beita = 0.0001
nita = 1
alpha = 0.00001
lambta = 0.1

In [9]:
#读入Xtemp，取需要部分组成X
Xtemp = np.zeros((390,128),dtype = float)
f = open('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/abstract2vec.txt')
lines = f.readlines()
Xtemp_row = 0
for line in lines:
    list = re.sub(' +',' ',line.strip('\n').strip()).split(' ')
    Xtemp[Xtemp_row][:] = list[0:128]
    Xtemp_row += 1
f.close()
print ("-----------abstract2vec.txt prepared-------------------")

-----------abstract2vec.txt prepared-------------------


In [10]:
#更新X，把它变为正
for i in range(n_init):
    for j in range(t):
        Xtemp[i][j] = Xtemp[i][j] + 1 

#组成X
X = np.zeros((352,128),dtype = float)
for i in range(n):
    for j in range(t):
        if 0 <= i and i < 100:
            X[i][j] = Xtemp[i+38][j]
        elif 100 <= i and i < 250:
            X[i][j] = Xtemp[i+75][j]
        elif 250 <= i and i <287 :
            X[i][j] = Xtemp[i-122][j]
        else:
            X[i][j] = Xtemp[i+38][j]

In [11]:
#读入A
f = open('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/authorauthor.txt')
A = np.ones((m,m))
lines = f.readlines()
A_row = 0
for line in lines:
    list = line.strip().split(' ')
    A[A_row:] = list[0:m]
    A_row += 1

f.close()
print ("-----------Author_author_matrix.txt prepared-------------------")

-----------Author_author_matrix.txt prepared-------------------


In [12]:
#读入Btemp
f = open('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/journalpaper.txt')
Btemp = np.ones((l,n_init))
lines = f.readlines()
Btemp_row = 0
for line in lines:
    list = line.strip().split(' ')
    Btemp[Btemp_row:] = list[0:n_init]
    Btemp_row += 1

f.close()
print ("-----------Journal_paper_matrix.txt prepared-------------------")

-----------Journal_paper_matrix.txt prepared-------------------


In [13]:
#组成B
B = np.ones((l,n))
for i in range(l):
    for j in range(n):
        if 0 <= j and j < 100:
            B[i][j] = Btemp[i][j+38]
        elif 100 <= j and j < 250:
            B[i][j] = Btemp[i][j+75]
        elif 250 <= j and j < 287:
            B[i][j] = Btemp[i][j-112]
        elif 187<= j and j < n:
            B[i][j] = Btemp[i][j+38]

In [14]:
Y = np.sign(A) # Y和A有关

In [15]:
#读入o
f = open('/Users/carina/Downloads/刘嘉莹/异常引用补充实验/sourcecode/ANCI_code/dataset/data/samejournal/matrix/IFS.txt')
o = np.ones((l,1))
lines = f.readlines()
o_row = 0
for line in lines:
    list = line.strip('\n')
    o[o_row] = list[0]
    o_row += 1

f.close()
print ("-----------Journals_IF.txt prepared-------------------")

-----------Journals_IF.txt prepared-------------------


In [16]:
yL = np.ones((r,1))   #真假标签,如果是1则是假
for i in range(r):
    if i>=0 and i < 100:
        yL[i][0] = -1
    else:
        yL[i][0] = 1

In [17]:
d = 3

In [18]:
#读入量
DL = np.ones((r,d))
DU = np.ones((n-r,d))
D = np.random.random(size = (n,d))
for i in range(r):
    for j in range(d):
        DL[i][j] = D[i][j]

for i in range(n-r):
    for j in range(d):
        DU[i][j] = D[i+r][j]

In [19]:
#随机初始化量
U = (np.random.random(size=(m,d)))
V = (np.random.random(size=(t,d)))
P = (-1 + 2*np.random.random(size=(d,1)))
#print(P)
Q = (-1 + 2*np.random.random(size=(d,1)))
T = (np.random.random(size=(d,d)))

L = calL()  #预算出L
L11 = np.zeros((m,m))
L12 = np.zeros((m,r))
L22 = np.zeros((r,r))
L21 = np.zeros((r,m))

for i in range(r):
    for j in range(r):
        L22[i][j] = L[m+i][m+j]

for i in range(r):
    for j in range(m):
        L21[i][j] = L[m+i][j]

for i in range(m):
    for j in range(m):
        L11[i][j] = L[i][j]

for i in range(m):
    for j in range(r):
        L12[i][j] = L[m][m+j]
            
print('----------到迭代了--------------')

390
2566
----------calculate L--------------
----------到迭代了--------------


In [20]:
iter = 7
for i in range(iter):
    print(i)
    #更新DL与DU
    yU = np.sign(np.dot(DU,P))
    #print(yU)
    evaluate(yU)
       
        
    D1 = np.dot(X,V) + gama*plus(np.dot(np.dot(OneZeroNorm(B).T,o),Q.T)) + gama*minus(np.dot(np.dot(np.dot(OneZeroNorm(B).T,OneZeroNorm(B)),D),np.dot(Q,Q.T)))+np.vstack(((nita*minus(np.dot(np.dot(DL,P),P.T)) + nita*plus(np.dot(yL,P.T)) +beita*minus(np.dot(L21,U))+ beita*minus(np.dot(L22,DL))), np.zeros((n-r,d))))

    D2 = np.dot(np.dot(D,V.T),V) + lambta*D + gama*plus(np.dot(np.dot(np.dot(OneZeroNorm(B).T,OneZeroNorm(B)),D),np.dot(Q,Q.T)))+ gama*minus(np.dot(np.dot(OneZeroNorm(B).T,o),Q.T))+ np.vstack(((beita*plus(np.dot(L21,U)) + beita*plus(np.dot(L22,DL)) + nita*plus(np.dot(np.dot(DL,P),P.T))+ nita*minus(np.dot(yL,P.T))), np.zeros((n-r,d))))

    #更新D
    for i in range(n):
        for j in range(d):
            D[i][j] = D[i][j] * math.sqrt(D1[i][j]/D2[i][j])

    for i in range(r):
        for j in range(d):
            DL[i][j] = D[i][j]

    for i in range(n-r):
        for j in range(d):
            DU[i][j] = D[i+r][j]
        
    U1 = alpha*(np.dot(np.dot(andor(Y,A),U),T.T)) + alpha*(np.dot(np.dot(andor(Y,A).T,U),T))+ beita*minus(np.dot(L11,U)) + beita*minus(np.dot(L12,DL))

        
    U2 = alpha*(np.dot(np.dot(andor(Y,np.dot(U,np.dot(T,U.T))),U),T.T)) + alpha*(np.dot(np.dot(andor(Y,np.dot(U,np.dot(T,U.T))).T,U),T))+ lambta*U + beita*plus(np.dot(L11,U)) + beita*plus(np.dot(L12,DL))
    #更新U
    for i in range(m):
        for j in range(d):
            U[i][j] = U[i][j] * math.sqrt(U1[i][j]/U2[i][j])
                
       

    #更新V
    for i in range(t):
        for j in range(d):
            V[i][j] = V[i][j] * math.sqrt(np.dot(X.T,D)[i][j]/(np.dot(np.dot(V,D.T),D)+lambta*V)[i][j])
        
        #更新T
    for i in range(d):
        for j in range(d):
            T[i][j] = T[i][j] * math.sqrt(np.dot(np.dot(alpha*U.T,andor(Y,A)),U)[i][j]/(np.dot(np.dot(alpha*U.T,andor(Y,np.dot(np.dot(U,T),U.T))),U) + lambta*T)[i][j])

       
    P = np.dot(np.linalg.inv(nita*np.dot(DL.T,DL) + lambta*np.eye(d,dtype = int)),np.dot(nita*DL.T,yL))
    Q = np.dot(np.linalg.inv(np.dot(np.dot(gama*np.dot(D.T,OneZeroNorm(B).T),OneZeroNorm(B)),D) + lambta*np.eye(d,dtype = int)),np.dot(np.dot(gama*D.T,OneZeroNorm(B).T),o))

            
yU = np.sign(np.dot(DU,P))
evaluate(yU)

0
1
1
64
36
Accuracy：0.6372549019607843
Precision：0.5
Recall：0.02702702702702703
F1：0.05128205128205129
1
0
0
65
37


ZeroDivisionError: float division by zero